### Preprocesamiento de los discusos - técnicas de desambiguación

In [3]:
# pip install spacy nltk scikit-learn transformers torch
# pip install es_core_news_sm
# python -m spacy download es_core_news_sm

In [1]:
import requests
import pandas as pd

In [2]:
import spacy


In [3]:
import pandas as pd
import nltk
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from typing import List, Dict, Any
from sklearn.model_selection import train_test_split



In [4]:
df_crude = pd.read_csv('../corpus/Manifesto_corpus_estructurado.csv')

In [5]:
indexPrueba = 4
print(df_crude['Text'][indexPrueba])
print(df_crude['Code'][indexPrueba],
df_crude['libertad_economica_score'][indexPrueba],
df_crude['libertad_personal_score'][indexPrueba])

Sin reglas de juego claras y justas en lo económico y en lo social, donde lo único permanente es el beneficio creciente de los sectores mas concentrados de ·la economía.
503 -1 1


In [6]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is

True

In [7]:
nlp = spacy.load("es_core_news_sm")



OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Las negaciones no deben ser consideradas stopwords ya que cambian el sentido de la oración.
negations = {"no", "nunca", "jamás", "nadie", "nada", "ninguno", "ninguna", "ni", "tampoco"}
stop_words = set(stopwords.words('spanish')) - negations
lemmatizer = WordNetLemmatizer()

In [ ]:
def limpiar_texto(texto: str) -> str:
    # Eliminar caracteres no alfanuméricos y convertir a minúsculas
    texto = re.sub(r'[^a-záéíóúñü\s]', '', texto.lower())
    return ' '.join(texto.split())

In [ ]:
def tokenizar(texto: str) -> List[str]:
    # Tokenizar el texto en palabras
    return word_tokenize(texto)

In [ ]:
def eliminar_stop_words(tokens: List[str]) -> List[str]:
    # Elimina palabras comunes que no aportan significado.
    return [token for token in tokens if token not in stop_words]

In [ ]:
def lematizar(tokens: List[str]) -> List[str]:
    # Reduce las palabras a su forma base
    doc = nlp(' '.join(tokens))
    # Evitar lemas inválidos como "economiar"
    return [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]


In [ ]:
import re

def desambiguar(texto: str) -> str:

    #Función que elimina indicios de ambigüedad, sarcasmo, ironía o figuras retóricas,
    #transformándolos en una versión más neutra sin perder el significado.


    # Diccionario de frases irónicas/sarcásticas y sus versiones neutras
    sarcasmo_neutro = {
        r"\bsí, claro\b": "obviamente no",       # "sí, claro" en tono irónico
        r"\bpor supuesto\b": "de ninguna manera", # "por supuesto" irónico
        r"\bqué sorpresa\b": "esto era esperado", # "qué sorpresa" sarcástico
        r"\bestoy seguro\b": "no estoy seguro",   # "estoy seguro" irónico
        r"\bno me digas\b": "es obvio",           # "no me digas" irónico
        r"\bqué gran trabajo\b": "mal trabajo",   # "qué gran trabajo" irónico
    }

    # Reemplazar frases irónicas por versiones neutras
    for patron, reemplazo in sarcasmo_neutro.items():
        texto = re.sub(patron, reemplazo, texto, flags=re.IGNORECASE)

    # Eliminar uso excesivo de signos de exclamación o interrogación
    texto = re.sub(r'[!?]+', '.', texto)

    # Eliminar comillas que pueden implicar ironía
    texto = re.sub(r'[\'\"]', '', texto)

    # Eliminar exageraciones comunes y normalizar las frases
    exageraciones = ['muy', 'sumamente', 'increíblemente', 'extremadamente', 'totalmente']
    for exag in exageraciones:
        texto = re.sub(rf'\b{exag}\b', '', texto, flags=re.IGNORECASE)

    # Eliminar frases que inviten a interpretación figurativa o sarcástica
    texto = re.sub(r"\b(debería ser)\b", "", texto)

    return texto.strip()

In [ ]:
from transformers import pipeline
# Cargar el modelo de clasificación de sentimientos en español
sentiment_classifier = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")


In [ ]:
import json
# Cargar patrones de sarcasmo desde el archivo JSON
with open('patrones_sarcasmo.json', 'r', encoding='utf-8') as f:
    sarcasm_data = json.load(f)
    sarcasm_patterns = sarcasm_data['sarcasm_patterns']
    sarcasm_keywords = sarcasm_data['sarcasm_keywords']

In [ ]:
def detect_sarcasm_patterns(text):
    """
    Detectar patrones de sarcasmo en una oración usando expresiones regulares.
    """
    for pattern in sarcasm_patterns:
        if re.search(pattern, text.lower()):
            return True
    return False

In [ ]:
def detect_sarcasm_keywords(doc, sentiment_label):
    """
    Detectar sarcasmo basado en palabras clave y el sentimiento.
    """
    for token in doc:
        if sentiment_label == 'NEG' and token.text.lower() in sarcasm_keywords['positivas']:
            return True
        if sentiment_label == 'POS' and token.text.lower() in sarcasm_keywords['negativas']:
            return True
    return False

In [ ]:
def reformulate_sentence(doc, sentiment_label):
    """
    Reformular una oración detectada como sarcástica.
    """
    reformulated = []
    sarcasm_detected = False

    for token in doc:
        if token.text.lower() in sarcasm_keywords["positivas"] and (sentiment_label == 'NEG' or detect_sarcasm_patterns(doc.text)):
            reformulated.append("no es " + token.text)
            sarcasm_detected = True
        elif token.text.lower() in sarcasm_keywords["negativas"] and (sentiment_label == 'POS' or detect_sarcasm_patterns(doc.text)):
            reformulated.append("no es tan " + token.text)
            sarcasm_detected = True
        else:
            reformulated.append(token.text)

    if sarcasm_detected:
        result = " ".join(reformulated)
        # Reformulaciones adicionales para sarcasmo común
        result = re.sub(r"(?i)\bclaro que sí\b", "no es cierto que", result)
        result = re.sub(r"(?i)\bqué gran\b", "no es tan gran", result)
        result = re.sub(r"(?i)\bobviamente\b", "no necesariamente", result)
        result = re.sub(r"(?i)\bseguramente\b", "posiblemente no", result)
        return result
    else:
        return doc.text

In [ ]:
def remove_sarcasm(text):
    """
    Detectar y reformular sarcasmo en un texto dado.
    """
    doc = nlp(text)
    sentiment = sentiment_classifier(text)[0]
    sarcasm_by_pattern = detect_sarcasm_patterns(text)
    sarcasm_by_keywords = detect_sarcasm_keywords(doc, sentiment['label'])
    if sarcasm_by_pattern or sarcasm_by_keywords:
        return reformulate_sentence(doc, sentiment['label'])
    else:
        return text

In [ ]:
def preprocesar_texto(texto: str) -> Dict[str, Any]:
    texto_limpio = limpiar_texto(texto)
    texto_sin_ambiguedad = remove_sarcasm(texto_limpio)
    tokens = tokenizar(texto_sin_ambiguedad)
    tokens_sin_stop = eliminar_stop_words(tokens)
    tokens_lematizados = lematizar(tokens_sin_stop)

    return ' '.join(tokens_lematizados)

In [ ]:
texto = df_crude.iloc[10, 0]
texto = "Que buen trabajo que estas haciendo!. Sigue así que solo lo empeoras más"
texto_preprocesado = preprocesar_texto(texto)
texto_preprocesado

'buen trabajo hacer seguir así solo empeora'

In [ ]:
print(df_crude['libertad_economica_score'].value_counts())

libertad_economica_score
 0    272578
-1     94401
 1     81118
Name: count, dtype: int64


In [ ]:
print(df_crude['libertad_personal_score'].value_counts())

libertad_personal_score
 0    346331
-1     58982
 1     42784
Name: count, dtype: int64


In [ ]:

# Función para obtener un subconjunto balanceado
def get_balanced_subset(df, column, n_per_category=50):
    balanced_df = pd.DataFrame()
    for category in [-1, 0, 1]:
        category_df = df[df[column] == category]
        if len(category_df) > n_per_category:
            category_df = category_df.sample(n=n_per_category, random_state=42)
        balanced_df = pd.concat([balanced_df, category_df])
    return balanced_df

# Obtener subconjuntos balanceados para ambas columnas
df_economica = get_balanced_subset(df_crude, 'libertad_economica_score')
df_personal = get_balanced_subset(df_crude, 'libertad_personal_score')

# Combinar los subconjuntos
df_balanced = pd.concat([df_economica, df_personal]).drop_duplicates()

# Verificar los recuentos
print(df_balanced['libertad_economica_score'].value_counts())
print(df_balanced['libertad_personal_score'].value_counts())

libertad_economica_score
 0    123
-1    118
 1     59
Name: count, dtype: int64
libertad_personal_score
 0    157
-1     77
 1     66
Name: count, dtype: int64


In [ ]:
df_balanced

,Text,Code,libertad_personal_score,libertad_economica_score
404498,Constitució d’un Consell Estatal de l’Habitatg...,504,-1,-1
434671,que incremente en forma acompasada su capacida...,503,1,-1
418994,Esto permitió reducir la tasa de deforestación...,416.2,0,-1
199449,"Redefinir la política social del país, para qu...",504,-1,-1
60719,Gracias a esa renegociación de los contratos p...,413,0,-1
...,...,...,...,...
231641,"Igualmente, en este contexto deben resaltarse ...",201.2,1,0
427797,Ninguna familia puede ser sometida a discrimin...,604,1,0
198355,Es un reto que debemos afrontar cuanto antes p...,503,1,-1
143802,"La desigualtat, a més de poder ser analitzada ...",503,1,-1


In [ ]:
df_balanced['text_processed'] = df_balanced['Text'].apply(preprocesar_texto)


In [ ]:
df_balanced.to_csv('df_balanced.csv', index=False)

In [ ]:
df_balanced

,Text,Code,libertad_personal_score,libertad_economica_score
404498,Constitució d’un Consell Estatal de l’Habitatg...,504,-1,-1
434671,que incremente en forma acompasada su capacida...,503,1,-1
418994,Esto permitió reducir la tasa de deforestación...,416.2,0,-1
199449,"Redefinir la política social del país, para qu...",504,-1,-1
60719,Gracias a esa renegociación de los contratos p...,413,0,-1
...,...,...,...,...
373447,que establezca las bases para el aprendizaje c...,201.2,1,0
188344,con vocación educativa en lo relativo a la igu...,503,1,-1
56025,Este programa pretende que personas en una con...,503,1,-1
84423,y se disminuirá la carga fiscal para los causa...,503,1,-1
